sourcecode_dep


In [ ]:
import networkx as nx
import json
import pandas as pd
import os
import matplotlib as plt
 
file_path = ''
excel_data = pd.read_excel(file_path)
dependency_graph = nx.DiGraph()

 
for _, row in excel_data.iterrows():
    source = row['name_version']
    target = row['Name_Version_file']
    
     
    dependency_graph.add_node(source, 
                              yuan=row['Source']
                             )
    dependency_graph.add_node(target, 
                              yuan1=row['Source_file'],
                              File=row['File'],
                              Code=row['Code'])

     
    dependency_graph.add_edge(target, source)

 
graph_data = nx.node_link_data(dependency_graph)

 
with open('./dependency_graph.json', 'w') as f:
    json.dump(graph_data, f, indent=4)

print('Graph has been saved to dependency_graph.json')


In [ ]:

weakly_connected_components = list(nx.weakly_connected_components(dependency_graph))
output_dir = ''
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

 
for i, component in enumerate(weakly_connected_components):
    if len(component) > 2:   
        subgraph = dependency_graph.subgraph(component).copy()

         
        subgraph_data = nx.node_link_data(subgraph)
        with open(os.path.join(output_dir, f'weakly_connected_subgraph_{i}.json'), 'w') as f:
            json.dump(subgraph_data, f, indent=4)

         
        plt.figure(figsize=(8, 6))
        pos = nx.spring_layout(subgraph)
        nx.draw(subgraph, pos, with_labels=True, node_color='lightgreen', font_weight='bold')
        plt.title(f'Weakly Connected Subgraph {i}')
        plt.savefig(os.path.join(output_dir, f'weakly_connected_subgraph_{i}.png'))
         
        plt.close()

print(f'Generated {len(weakly_connected_components)} weakly connected subgraphs and saved to {output_dir}')


metadata_dep

In [ ]:
import pandas as pd
import networkx as nx
import json
from tqdm import tqdm
import ast


def handle_non_serializable(obj):
    if isinstance(obj, set):
        return list(obj)
    elif isinstance(obj, nx.classes.reportviews.NodeDataView):
        return list(obj)
     

def append_kg_to_json(graph, filename):
    json_data = nx.node_link_data(graph)
    json_str = json.dumps(json_data, indent=4, default=handle_non_serializable)
    with open(filename, 'a', encoding='utf-8') as json_file:
        json_file.write(json_str)
        json_file.write("\n")   

def bulid_kg():
     
     
    excel_path = ''   
    sheet_name = ''         
     
    df = pd.read_excel(excel_path,sheet_name=sheet_name)
    graph = nx.Graph()
    node_count = 0

     
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Building Knowledge Graph"):
         
        node_data = {
            "package_id": row['ID'],
            "OSS": row['OSS'],
            "Source": row['Source'],
            "Status": row['Status'],
            "filepath": row['Filepath'],
            "package_name": row['name_version'],
            "name":row['name'],
            "verison":row['version'],
            "published_at":row['published_at'],
            "description":row['description'],
            "author":row['author'],
            "maintainers":row['maintainers'],
            "repository_url":row['repository_url'],
            "dependencies":row['dependencies'],


            "edge_dependency": list()

        }

         
        for key, value in node_data.items():
            if pd.isna(value):
                node_data[key] = None
         
        graph.add_node(row['ID'], **node_data)
        node_count += 1
        
        dependency_list = node_data['dependencies'].split(', ') if node_data['dependencies'] else []

         
        for other_index, other_row in df.iterrows():
            if index != other_index:
                other_id = other_row['ID']
                other_name = other_row['name']

                 
                current_name = row['name']

                 
                if pd.notna(other_name) and other_name != current_name and any(dep.strip() == other_name for dep in dependency_list):
                    graph.add_edge(row['ID'], other_id)
                    graph.nodes[row['ID']].setdefault('edge_dependency', []).append(other_id)


     
    json_data = nx.node_link_data(graph)
    json_str = json.dumps(json_data, indent=4, default=handle_non_serializable)
    with open('', 'w', encoding='utf-8') as json_file:
        json_file.write(json_str)
        new_source_publish = None if pd.isna(new_source_publish) or new_source_publish == '' else new_source_publish

bulid_kg()
 

In [ ]:
import json
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm   

 
with open('', 'r') as json_file:
    graph_data = json.load(json_file)
    knowledge_graph = nx.node_link_graph(graph_data)

 
connected_components = list(nx.connected_components(knowledge_graph))

 
for i, component in tqdm(enumerate(connected_components, 1), total=len(connected_components), desc="Processing Subgraphs"):
     
    if len(component) > 1:
        subgraph = knowledge_graph.subgraph(component)
        
         
        subgraph_data = nx.node_link_data(subgraph)
        subgraph_json_str = json.dumps(subgraph_data, indent=4)

        with open(f'subgraph_{i}.json', 'w') as subgraph_json_file:
            subgraph_json_file.write(subgraph_json_str)

         
        plt.figure(figsize=(20, 10))
        pos = nx.spring_layout(subgraph)   
        nx.draw(subgraph, pos, with_labels=True, node_color='skyblue', node_size=500, font_size=8)
        plt.title(f"SubGraph {i}")
         
        plt.savefig(f'subgraph_{i}.pdf', format='pdf', bbox_inches='tight')
         


Two types of subgraph deduplication

In [ ]:
 
import os
import json

def load_json_files(folder_path):
    json_files = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                json_data = json.load(file)
                json_files[file_name] = json_data
    return json_files

def compare_nodes(nodes1, nodes2):
    names1 = {node['package_name'] for node in nodes1}
    names2 = {node['id'] for node in nodes2}
    return names1.issubset(names2)

def find_matching_jsons(folder1_jsons, folder2_jsons):
    matching_json_pairs = []
    
    for file1_name, data1 in folder1_jsons.items():
        nodes1 = data1.get('nodes', [])
        
        for file2_name, data2 in folder2_jsons.items():
            nodes2 = data2.get('nodes', [])
            
            if compare_nodes(nodes1, nodes2):
                matching_json_pairs.append((file1_name, file2_name))
    
    return matching_json_pairs

def main(folder1, folder2):
    folder1_jsons = load_json_files(folder1)
    folder2_jsons = load_json_files(folder2)
    
    matching_json_pairs = find_matching_jsons(folder1_jsons, folder2_jsons)
    
    if matching_json_pairs:
        for file1_name, file2_name in matching_json_pairs:
            print(f"File '{file1_name}' from Folder 1 matches with File '{file2_name}' from Folder 2.")
    else:
        print("No matching JSON files found.")
    return matching_json_pairs

 
folder1_path = ''
folder2_path = ''   

matching_json_pairs = main(folder1_path, folder2_path)
